In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
%run /Workspace/Users/evansavo@gmail.com/Population_Health_&_Readmission_Risk/1.setup/utilities

In [0]:
# setup dbutils
dbutils.widgets.text('catalog', 'phr', 'Catalog')
dbutils.widgets.text('data_source', '', 'Data Source')

In [0]:
catalog = dbutils.widgets.get('catalog')
data_source = dbutils.widgets.get('data_source')

base_path = f'databricks_cms_synthetic_public_use_files_synpuf.cms_synpuf_ext.{data_source}'


print(base_path)

# Bronze level

In [0]:
# Load table
df = spark.read.table(base_path).withColumn('read_timestamp', F.current_timestamp())

display(df.limit(2))

In [0]:
# Save data
data_source = 'pd_events'
(df.write
 .format('delta')
 .option('delta.enableChangeDataFeed', 'true')
 .mode('overwrite')
 .saveAsTable(f'{catalog}.{bronze_schema}.{data_source}')
)

# Silver Level

In [0]:
silver_df = spark.sql(f'select * from {catalog}.{bronze_schema}.{data_source}')
# display(silver_df.limit(3))

In [0]:
# Clean SRVC_DT
silver_df = silver_df.withColumn('SRVC_DT', F.to_date(F.col('SRVC_DT'), 'yyyyMMdd'))

# Change Dtype quantities
silver_df = silver_df.withColumn('QTY_DSPNSD_NUM', F.col('QTY_DSPNSD_NUM').cast('float').cast('int'))
silver_df = silver_df.withColumn('DAYS_SUPLY_NUM', F.col('DAYS_SUPLY_NUM').cast('float').cast('int'))

# Change Dtype amounts
silver_df = silver_df.withColumn('PTNT_PAY_AMT', F.col('PTNT_PAY_AMT').cast('double'))
silver_df = silver_df.withColumn('TOT_RX_CST_AMT', F.col('TOT_RX_CST_AMT').cast('double'))

In [0]:
# Change Dtype
silver_df = silver_df.withColumn('QTY_DSPNSD_NUM', F.col('QTY_DSPNSD_NUM').cast('float'))
silver_df = silver_df.withColumn('DAYS_SUPLY_NUM', F.col('DAYS_SUPLY_NUM').cast('float'))
silver_df = silver_df.withColumn('PTNT_PAY_AMT', F.col('PTNT_PAY_AMT').cast('float'))
silver_df = silver_df.withColumn('TOT_RX_CST_AMT', F.col('TOT_RX_CST_AMT').cast('float'))

In [0]:
# Drop read_timestamp
silver_df = silver_df.drop("read_timestamp")

In [0]:
# Load ben_sum
ben_sum = spark.read.table('phr.`02_silver`.ben_sum')

# Drop read_timestamp
ben_sum = ben_sum.drop("read_timestamp")

# Join with ben_sum
silver_df = silver_df.join(ben_sum, on='DESYNPUF_ID', how='inner')

In [0]:
# Write to silver schema
(  silver_df.write
    .format('delta')
    .mode('overwrite')
    .option('enableChangeDataFeed', 'true')
    .saveAsTable(f'{catalog}.{silver_schema}.{data_source}_enriched')
)